<center>
    <h2>Online learning platform database - Cassandra</h2>
    <h3>Performing the queries and storing the queries execution time</h3>
</center>

<h3>Python - Cassandra interaction</h3>

Prior to performing the queries we import the required modules (the Cassandra Python driver and the <i>time</i> and <i>csv</i> modules), establish a connection with the Cassandra instance running in Docker and choose the keyspace on which we will perform the queries.

In [1]:
from cassandra.cluster import Cluster   # MySQL driver
import time                             # time-related functions to register query execution times
import csv                              # read and write csv files

# instantiate a cluster
cluster = Cluster(['127.0.0.1'])

# create a session by connecting to the cluster
session = cluster.connect()

# associate a keyspace to the session
session.set_keyspace('dbb_cassandra_test')

<h3>Query the datasets</h3>

I create a dictionary of lists for each of the four keyspaces. In these dictionaries the keys are the query names and the values are the 31 query execution times: in fact I attach the value of the query execution time of the most recent query to the list. Since query execution times are required in milliseconds, prior to attaching them, I multiply them by 1000 and round them to the fifth decimal precision.
The above summarized actions (for each of the four queries on each of the four keyspaces) are performed by following a standard succession of steps. Each step is encapsulated within a notebook cell (so each query is performed 31 times by using three notebook cells), as follows:
 - step 1: create index if needed, define the query, perform it for the first time, contextually create timestamps prior and after query execution, print query result;
 - step 2: compute execution time of the first query execution and store it within the corresponding dictionary list;
 - step 3: [thirty times] perform query execution while creating prior and following timestamps, compute execution time and store it within the corresponding dictionary list <u>, reset the cursor to allow repeating the query</u>.

For each dataset, after having performed the four queries, I will finally compute the mean of the query executions from step 3. Together with the first query execution, this mean value will be stored into a new dictionary, specific to a dataset. Originally, I would use these four new dictionaries to save the query execution times into a csv file for constructing histograms. I later resolved to save all the 31 recorded query execution times and pass them all to Microsoft© Excel to process them.

In [2]:
smallDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
mediumDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
largeDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
humongousDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}

In [14]:
# mean function (used to compute mean execution time of the 30 grouped queries)
def mean(aList):
    n = len(aList)
    sum = 0
    for value in aList:
        sum += value
    return sum / n

<h3>Table with 250k records</h3><br>
I start with the smallest table.

<h4>Query1</h4>
It is to be noted that, in relation to its focus on performance, Cassandra does not allow the use of the <code>DISTINCT</code> keyword on columns that are not partition keys (primary keys). In <i>Query 1</i>, however, we are only interested in the names of the students enrolled to the course having ID = 192, there is no point in having a student's name repeated more than once. Each student enrolled to course 192, however, has accessed to various learning materials of course 192, hence the query result, without the use of <code>DISTINCT</code> will be a <i>bag</i>, rather than a <i>set</i>. Duplicate values must be handled in some way, in order to display each student only once. I have considered two possibilities to reach the desired result:

- the first one uses CQLSH and an external csv file. It requires, within the selected keyspace, creating a new table with three fields (student first name, student last name and course ID) and setting the first two as primary key. Then the same three fields are to be copied from the entire original table to a csv file and copied back from the csv file to the table having firstname and lastName as primary key. Then the query on the new table can be run by using the <code>DISTINCT</code> option:

    <code>
        'CREATE TABLE query1_temp (firstName VARCHAR, lastName VARCHAR, courseID VARCHAR, PRIMARY KEY(firstName, lastName));'
        'COPY smallDB(firstName, lastName, courseID) TO 'path/query1_temp.csv' WITH HEADER = TRUE AND DELIMITER = ',';'
        'COPY query1_temp(firstname, lastName, courseID) FROM 'path/query1_temp.csv' WITH HEADER = TRUE AND DELIMITER = ',';'
        'SELECT DISTINCT firstName, lastName FROM query1_temp WHERE courseID = 192;'
    </code>

- the second one just considers performing the query on the original table without the use of the <code>DISTINCT</code> keyword. The query result is then processed via programming language to obtain the unique values of the students enrolled to course 192. Python is suitable for this purpose, having in store set objects than do not allow element replicas.

Both methods will affect the query execution time, if all the steps are to be taken into account. The second method seems to me the cleanest one and I will apply it for <i>Query 1</i>. In particular, I will add to <i>step 1</i> a new substep: prior to recording timestamps I create a set (<i>resSet</i>) where I want to store unique values from the query result. After having defined thid object and created the index required by the query: I record the first timestamp, I run the query, I manipulate the query result by storing the rows into the <i>resSet</i> object, I record the last timestamp. Then the query result can be displayed.

In [36]:
# step 1
resSet1 = set()
session.execute('CREATE INDEX IF NOT EXISTS query1Index ON smalldb(courseid);')
small_cassandra1 = 'SELECT firstName, lastName FROM smalldb WHERE courseid = \'192\';'

before = time.time()
small_query1 = session.execute(small_cassandra1)
for row in small_query1:
    resSet1.add(row)
after = time.time()

for element in resSet1:
    print(element[0], element[1])

Casandra Arenas
Ledün Soylu
Patrícia Leite
Vigilija Gaižauskas
Narciso Ferrán
Sarah Lara
Nath Nicolas
Cathrine Lie
Custodia Hidalgo
Émile Nicolas
Ana Narušis
Ingeborg Amundsen
Arthur Laroche


In [37]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query1'].append(round(msec_duration, 5))

In [39]:
# step 3
for i in range(0, 30):
    resSet1 = set()
    before = time.time()
    small_query1 = session.execute(small_cassandra1)
    for row in small_query1:
        resSet1.add(row)
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query1'].append(round(msec_duration, 5))

In [27]:
smallDict

{'query1': [82.04389,
  75.12617,
  47.38998,
  43.33997,
  37.53185,
  50.21596,
  58.46024,
  53.57504,
  44.61384,
  56.72574,
  67.25574,
  62.52623,
  74.18704,
  54.96001,
  38.90896,
  55.60422,
  50.2069,
  54.97217,
  49.22771,
  43.29991,
  64.61501,
  41.8992,
  40.11822,
  41.27002,
  45.6481,
  58.4147,
  51.157,
  61.58209,
  57.70183,
  63.97605,
  66.19596],
 'query2': [],
 'query3': [],
 'query4': []}

<h4>Query 2</h4>
<i>Query 2</i> requires selection on more than one field value (discipline must be 'statistics' and the year of the course must be '2022'. In this case two indices must be created, but Cassandra requires the <code>ALLOW FILTERING</code> clause because double indexing may negatively impact query performance. Even in this case we need unique values, hence I add results to a Python set after query completion.

In [28]:
# step 1
resSet2 = set()
session.execute('CREATE INDEX IF NOT EXISTS query2Index1 ON smalldb(discipline);')
session.execute('CREATE INDEX IF NOT EXISTS query2Index2 ON smalldb(courseyear);')
small_cassandra2 = 'SELECT coursename FROM smalldb WHERE discipline = \'statistics\' AND courseyear = \'2022\' ALLOW FILTERING;'

before = time.time()
small_query2 = session.execute(small_cassandra2)
for row in small_query2:
    resSet2.add(row)
after = time.time()

for element in resSet2:
    print(element[0])

Exploratory Data Analysis
Introduction to Probability and Data with R
Bayesian Statistics: From Concept to Data Analysis
Foundations: Data, Data, Everywhere
Econometrics: Methods and Applications
Basic Statistics
Introduction to Statistics
Python and Statistics for Financial Analysis
Understanding Clinical Research: Behind the Statistics


In [29]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query2'].append(round(msec_duration, 5))

In [30]:
# step 3
for i in range(0, 30):
    before = time.time()
    small_query2 = session.execute(small_cassandra2)
    for row in small_query2:
        resSet2.add(row)
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query2'].append(round(msec_duration, 5))

In [31]:
smallDict

{'query1': [82.04389,
  75.12617,
  47.38998,
  43.33997,
  37.53185,
  50.21596,
  58.46024,
  53.57504,
  44.61384,
  56.72574,
  67.25574,
  62.52623,
  74.18704,
  54.96001,
  38.90896,
  55.60422,
  50.2069,
  54.97217,
  49.22771,
  43.29991,
  64.61501,
  41.8992,
  40.11822,
  41.27002,
  45.6481,
  58.4147,
  51.157,
  61.58209,
  57.70183,
  63.97605,
  66.19596],
 'query2': [355.6149,
  300.19498,
  307.863,
  290.27891,
  306.96201,
  286.4759,
  327.95,
  331.97594,
  303.18809,
  347.83101,
  315.45591,
  340.63697,
  329.01716,
  469.95592,
  283.3631,
  346.22002,
  298.14911,
  349.04981,
  321.83909,
  351.92919,
  296.23199,
  314.04996,
  307.90281,
  308.39276,
  287.75907,
  308.87985,
  290.93719,
  304.68106,
  300.45891,
  304.76594,
  408.48303],
 'query3': [],
 'query4': []}

<h4>Query 3</h4>
In this case the hardest difficulty was in trying to implement an index that could behave similarly to the MySQL <code>LIKE</code>, by matching patterns in string. In Cassandra custom indices can be created, in particular the so-called <a href = 'https://cassandra.apache.org/doc/stable/cassandra/cql/SASI.html'>SASI indexes</a> which can be set on three different modes: <code>PREFIX</code> (default), <code>CONTAINS</code> or <code>SPARSE</code>.<br>
<br>
The first one allows use of a syntax such as the following:<br>
<code>SELECT <i>fieldNames</i> WHERE <i>fieldName</i> LIKE '<i>prefix</i>%;'</code><br>
<br>
The second one would allow either suffixes or strings contained in another string:
<br>
<code>SELECT <i>fieldNames</i> WHERE <i>fieldName</i> LIKE '%<i>contained</i>%;'</code><br>
or
<br>
<code>SELECT <i>fieldNames</i> WHERE <i>fieldName</i> LIKE '%<i>suffix</i>;'</code><br>

As for the <code>SPARSE </code> mode, I just found <a href = 'https://www.doanduyhai.com/blog/?p=2058'>here</a> some details. This mode is mainly designed for cases when very few occurrences match the query.<br>
Based on this, I approached the definition of <i>Query 3</i> as per the following cell. I create two indices working on the <i>discipline</i> and on the <i>material type</i> and another custom index for the <i>email</i> field. Then I build the <code>WHERE</code> clause on the three indices.

In [72]:
# set the indices
session.execute('CREATE INDEX IF NOT EXISTS query3Index1 ON smalldb(discipline);')
session.execute('CREATE INDEX IF NOT EXISTS query3Index2 ON smalldb(materialtype);')
session.execute('CREATE CUSTOM INDEX IF NOT EXISTS SASIquery3Index3 ON smalldb(email) USING \'org.apache.cassandra.index.sasi.SASIIndex\' WITH OPTIONS = {\'mode\': \'CONTAINS\', \'analyzer_class\': \'org.apache.cassandra.index.sasi.analyzer.NonTokenizingAnalyzer\', \'case_sensitive\': \'false\'};')

# define the CQL query
test = 'SELECT materialid FROM smalldb WHERE discipline = \'maths\' AND materialtype = \'lecture slides\' AND email LIKE \'%gmail.com\' ALLOW FILTERING;'

# run the query
smallqueryTest_query3 = session.execute(test)

ReadFailure: Error from server: code=1300 [Replica(s) failed to execute read] message="Operation failed - received 0 responses and 1 failures: UNKNOWN from /172.25.0.2:7000" info={'consistency': 'LOCAL_ONE', 'required_responses': 1, 'received_responses': 0, 'failures': 1, 'error_code_map': {'172.25.0.2': '0x0000'}}

A <i>ReadFailure error</i> is thrown and I believe it is associated to the use of a custom index together with two regular indices. In fact, by executing two separate queries with the regular indices or the custom one, results are obtained.

In [73]:
test1 = 'SELECT materialid FROM smalldb WHERE discipline = \'maths\' AND materialtype = \'lecture slides\' ALLOW FILTERING;'
queryTest1 = session.execute(test1)
for i in range(0, 10):
    print(queryTest1[i])

Row(materialid='21516')
Row(materialid='4163')
Row(materialid='21576')
Row(materialid='21308')
Row(materialid='4328')
Row(materialid='21329')
Row(materialid='21370')
Row(materialid='21490')
Row(materialid='21367')
Row(materialid='21727')


/var/folders/yb/85lnwyqj61q7l7rp510bt7340000gn/T/ipykernel_2237/435765169.py:4: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  print(queryTest1[i])


In [74]:
test2 = 'SELECT materialid FROM smalldb WHERE email LIKE \'%gmail.com\' ALLOW FILTERING;'
queryTest2 = session.execute(test2)
for i in range(0, 10):
    print(queryTest2[i])

/var/folders/yb/85lnwyqj61q7l7rp510bt7340000gn/T/ipykernel_2237/1889992488.py:4: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  print(queryTest2[i])


Row(materialid='17036')
Row(materialid='9497')
Row(materialid='11515')
Row(materialid='9388')
Row(materialid='19901')
Row(materialid='23612')
Row(materialid='8111')
Row(materialid='13673')
Row(materialid='11260')
Row(materialid='18429')


I then resolved to run a more simplified query and manipulate the result via programming language to obtain the desired result. I get the rows where the discipline is <i>maths</i> and the learning material type is <i>lecture slides</i>, then I run a custom function on the query result. The function allows to obtain the domain of the email in the query result, this allows me to count only those matching the string '<i>gmail.com</i>'. In this case, all the query results are of interest, since we want to know how many learning materials have been accessed by students, irrespective of possible repetitions in accessed learning materials.

In [98]:
# define function taking an email string and returning a substring with the email domain
def findDomain(email):
    delimiter = '@'
    emailList = email.split(delimiter)
    return emailList[1]

In [118]:
# step 1
small_cassandra3 = 'SELECT email FROM smalldb WHERE discipline = \'maths\' AND materialtype = \'lecture slides\' ALLOW FILTERING;'

before = time.time()
small_query3 = session.execute(small_cassandra3)
counter = 0
for row in small_query3:
    if findDomain(row.email) == 'gmail.com':
        counter += 1
    else:
        counter += 0
after = time.time()

print(counter)

837


In [119]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query3'].append(round(msec_duration, 5))

In [120]:
# step 3
for i in range(0, 30):
    before = time.time()
    small_query3 = session.execute(small_cassandra3)
    counter = 0
    for row in small_query3:
        if findDomain(row.email) == 'gmail.com':
            counter += 1
        else:
            counter += 0
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query3'].append(round(msec_duration, 5))

In [121]:
smallDict

{'query1': [],
 'query2': [],
 'query3': [349.41912,
  301.29671,
  461.88903,
  223.49715,
  248.20733,
  268.64219,
  229.48599,
  273.98896,
  226.70722,
  250.63014,
  255.47624,
  267.39931,
  258.90899,
  244.13705,
  251.39594,
  237.3991,
  237.19001,
  227.19789,
  220.70694,
  233.03795,
  250.90599,
  247.4668,
  238.18326,
  249.78209,
  482.79309,
  297.97983,
  232.23805,
  436.58423,
  233.19674,
  225.93164,
  229.34413],
 'query4': []}

<h4>Query 4</h4>
<i>Query 4</i> presents analogous problems to those of <i>Query 3</i> since multiple occurrences of Korean countries are present in the table (South Korea, North Korea, Républica de Corea, etc.). In this case, to achieve the desired task, instead of trying to using a custom index, I preferred to exploit the <code>IN</code> set operator, by using it on the complete list of occurrences of Korean countries. In this way, only students from a country present in the limited set of Korean countries can be selected (together with those enrolled to a course of the discipline '<i>psychology</i>'). Considering that birthdate is simply another string value and given that string manipulation or pattern searches would require using a custom index together with other selecting approaches, which would raise the already experienced problems, choosing students born before year 2000 seems a  hard task, which, given my configuration, would probably be better achieved via programming language. So I define a function to extract the year from the <i>dateofbirth</i> field and check if the year precedes year 2000. I use the function on the elements of the Python set to which I have already added the query results, which seems a more efficient approach than using the function on the query result and later adding surviving results in a Python set.

In [150]:
# define function taking a birthdate string in the format yyyy-mm-dd and returning a substring with the year
def findYear(dateofbirth):
    delimiter = '-'
    dateList = dateofbirth.split(delimiter)
    return int(dateList[0])

In [174]:
# step 1
resSet4 = set()
session.execute('CREATE INDEX IF NOT EXISTS query4Index1 ON smalldb(discipline);')
small_cassandra4 = 'SELECT firstname, lastname, country, dateofbirth FROM smalldb WHERE discipline = \'psychology\' AND country IN (\'Korea\', \'República de Corea\', \'South Korea\', \'North Korea\', \'República Popular Democrática de Corea\', \'Sydkorea\') ALLOW FILTERING;'

before = time.time()
small_query4 = session.execute(small_cassandra4)
for row in small_query4:
    if findYear(row.dateofbirth) < 2000:
        resSet4.add(row)
after = time.time()

for element in resSet4:
    print(element.firstname, element.lastname, element.country)

Cathrine Lie South Korea
Raghav Sura North Korea
Lynda Reynolds Korea


In [175]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query4'].append(round(msec_duration, 5))

In [176]:
# step 3
for i in range(0, 30):
    before = time.time()
    small_query4 = session.execute(small_cassandra4)
    for row in small_query4:
        resSet4.add(row)
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query4'].append(round(msec_duration, 5))

In [177]:
smallDict

{'query1': [],
 'query2': [],
 'query3': [349.41912,
  301.29671,
  461.88903,
  223.49715,
  248.20733,
  268.64219,
  229.48599,
  273.98896,
  226.70722,
  250.63014,
  255.47624,
  267.39931,
  258.90899,
  244.13705,
  251.39594,
  237.3991,
  237.19001,
  227.19789,
  220.70694,
  233.03795,
  250.90599,
  247.4668,
  238.18326,
  249.78209,
  482.79309,
  297.97983,
  232.23805,
  436.58423,
  233.19674,
  225.93164,
  229.34413],
 'query4': [695.7531,
  635.82397,
  750.34499,
  712.49294,
  661.04388,
  776.11971,
  668.58721,
  652.04597,
  628.80802,
  687.53362,
  648.87118,
  629.42886,
  686.27191,
  590.88993,
  626.34301,
  655.761,
  616.15419,
  822.73889,
  710.8779,
  670.41206,
  1154.93011,
  747.22934,
  931.73409,
  883.31389,
  694.96393,
  785.81524,
  923.58875,
  1341.88581,
  600.34895,
  739.90393,
  705.01399]}

<h3>Table with 500k records</h3><br>

<h4>Query1</h4>

In [31]:
# step 1
resSet1 = set()
session.execute('CREATE INDEX IF NOT EXISTS med_query1Index ON mediumdb(courseid);')
medium_cassandra1 = 'SELECT firstName, lastName FROM mediumdb WHERE courseid = \'192\';'

before = time.time()
medium_query1 = session.execute(medium_cassandra1)
for row in medium_query1:
    resSet1.add(row)
after = time.time()

for element in resSet1:
    print(element[0], element[1])

Nath Nicolas
Narciso Ferrán
Karl Christensen
Débora Vaz
Yuvaan Dara
Ana Narušis
Ingeborg Amundsen
Ledün Soylu
Casandra Arenas
Joris Kavaliauskas
Custodia Hidalgo
Cathrine Lie
Émile Nicolas
Christl Henschel
Arthur Laroche
Patrícia Leite
Vigilija Gaižauskas
Sarah Lara
Miguel Real
Nedas Naujokas


In [32]:
# step 2
msec_duration = (after - before) * 1000
mediumDict['query1'].append(round(msec_duration, 5))

In [41]:
# step 3
for i in range(0, 30):
    resSet1 = set()
    before = time.time()
    medium_query1 = session.execute(medium_cassandra1)
    for row in medium_query1:
        resSet1.add(row)
    after = time.time()
    msec_duration = (after - before) * 1000
    mediumDict['query1'].append(round(msec_duration, 5))

In [42]:
mediumDict

{'query1': [145.22409,
  92.65804,
  81.97904,
  79.16284,
  74.36013,
  104.69818,
  137.92205,
  117.68174,
  109.49111,
  91.62211,
  89.98489,
  75.12617,
  65.26208,
  64.42904,
  64.4331,
  64.71801,
  60.58002,
  66.72716,
  86.42983,
  122.89691,
  73.32587,
  78.81594,
  70.5359,
  63.82418,
  72.45398,
  113.89518,
  95.33811,
  63.55691,
  60.77981,
  67.7979],
 'query2': [],
 'query3': [],
 'query4': []}